In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class IndianFacesDataset(Dataset):
    def __init__(self, img_dir, transform=None, race_filter=3):
        self.img_dir = img_dir
        self.transform = transform
        
        # Check if folder exists and list files
        print(f"Looking in folder: {img_dir}")
        if not os.path.exists(img_dir):
            print(f"Error: Folder {img_dir} does not exist!")
        
        self.img_names = [f for f in os.listdir(img_dir) if f.endswith('jpg.chip.jpg')]
        print(f"Found {len(self.img_names)} files ending with '.jpg.chip'")
        
        # Parse metadata and filter for Indian (race=3)
        self.metadata = []
        for img_name in self.img_names:
            parts = img_name.split("_")
            try:
                age, gender, race = int(parts[0]), int(parts[1]), int(parts[2])
                if race == race_filter:
                    self.metadata.append({"filename": img_name, "age": age})
            except (IndexError, ValueError) as e:
                print(f"Skipping {img_name}: Invalid format ({e})")
        
        print(f"Found {len(self.metadata)} Indian images (race=3)")

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_info = self.metadata[idx]
        img_path = os.path.join(self.img_dir, img_info["filename"])
        image = Image.open(img_path).convert("RGB")
        age = torch.tensor(img_info["age"], dtype=torch.float32) / 116.0
        if self.transform:
            image = self.transform(image)
        return {"image": image, "age": age}

# Define transforms
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Pass your folder path
folder_path = r"C:\Users\ajinf\Documents\CS 7150\FaceAgingDFRL\image_dataset\indian_images"  # Replace with your actual path
dataset = IndianFacesDataset(folder_path, transform=transform, race_filter=3)
print(f"Dataset size: {len(dataset)}")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

Looking in folder: C:\Users\ajinf\Documents\CS 7150\FaceAgingDFRL\image_dataset\indian_images
Found 3975 files ending with '.jpg.chip'
Found 3975 Indian images (race=3)
Dataset size: 3975


In [2]:
from diffusers import StableDiffusionPipeline
import torch

# Load pretrained LDM
pipe = StableDiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16)
vae = pipe.vae.to("cuda")
unet = pipe.unet.to("cuda")  # We’ll fine-tune this
scheduler = pipe.scheduler

C:\Users\ajinf\Documents\CS 7150\FaceAgingDFRL\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ajinf\Documents\CS 7150\FaceAgingDFRL\.venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ajinf\.cache\huggingface\hub\models--stable-diffusion-v1-5--stable-diffusion-v1-5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to r

AssertionError: Torch not compiled with CUDA enabled